In [1]:
#This project goes over a final project in my Machine Learning II class at Georgetown University. It covers PCA and Clustering techniques.
#Unfortunately Kaggle does not support some of the packages required for the script to fully function outside of a local environment. I can share the R file if necessary.
#I have commented out the code so that it can at least be viewed on this forum.
#A report also accompanied this code which goes deeper into the thought process behind the analysis.

In [2]:
install.packages("xlsx")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘rJava’, ‘xlsxjars’




In [3]:
#### Load Packages ####
library(xlsx)
library(reshape2)
library(dplyr)
library(tidyverse)
library(caret)
# library(corrplot)
# library(patchwork)
# library(factoextra) 
# library(areaplot)
# library(FactoMineR)
# library(factoextra)
# library(fastICA)
# library(tsne)
# library(car)
# library(NbClust) 
# library(clValid) 


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.2     ✔ stringr   1.5.0
✔ lubridate 1.9.2     ✔ tibble    3.2.1
✔ purrr     1.0.1     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


The following object is masked from ‘package:httr’:

    progress




In [4]:
#### Data ingestion ####
travelDataOrig= readxl::read_xlsx("/kaggle/input/travel-reviews/Travel_Review.xlsx")

#Remove UserID
travelData <- travelDataOrig %>%
    select(-UserID)


#change character columns to numeric
str(travelData)
travelData <- data.frame(sapply(travelData, as.numeric))
str(travelData)

#check nas
sapply(travelData, function(x){sum(is.na(x))})

#gardens has one null - replacing it with the median
travelData[is.na(travelData$Gardens),'Gardens'] = median(travelData$Gardens,na.rm=TRUE) 

sapply(travelData, function(x){sum(is.na(x))})
#No nulls!

summary(travelData)
dim(travelData)

#### Check presence of zeroes ####
sapply(travelData, function(x){sum(x == 0)})
# A ton of zeroes 

Warning message:
“Expecting numeric in L2714 / R2714C12: got '2  2.'”
New names:
• `` -> `...26`


tibble [5,456 × 25] (S3: tbl_df/tbl/data.frame)
 $ Churches            : num [1:5456] 0 0 0 0 0 0 0 0 0 0 ...
 $ Resorts             : num [1:5456] 0 0 0 0.5 0 0 5 5 5 5 ...
 $ Beaches             : num [1:5456] 3.63 3.63 3.63 3.63 3.63 3.63 3.63 3.63 3.64 3.64 ...
 $ Parks               : num [1:5456] 3.65 3.65 3.63 3.63 3.63 3.63 3.63 3.63 3.64 3.64 ...
 $ Theatres            : num [1:5456] 5 5 5 5 5 5 5 5 5 5 ...
 $ Museums             : num [1:5456] 2.92 2.92 2.92 2.92 2.92 2.92 2.92 2.92 2.92 2.92 ...
 $ Malls               : num [1:5456] 5 5 5 5 5 5 3.03 5 3.03 5 ...
 $ Zoo                 : num [1:5456] 2.35 2.64 2.64 2.35 2.64 2.63 2.35 2.63 2.62 2.35 ...
 $ Restaurants         : num [1:5456] 2.33 2.33 2.33 2.33 2.33 2.33 2.33 2.33 2.32 2.32 ...
 $ Pubs_Bars           : num [1:5456] 2.64 2.65 2.64 2.64 2.64 2.65 2.64 2.64 2.63 2.63 ...
 $ LocalServices       : num [1:5456] 1.7 1.7 1.7 1.73 1.7 1.71 1.73 1.7 1.71 1.69 ...
 $ Burger_PizzaShops   : num [1:5456] 1.69 1.69 1.69 1.69

Churches              Resorts              Beaches 
                   0                    0                    0 
               Parks             Theatres              Museums 
                   0                    0                    0 
               Malls                  Zoo          Restaurants 
                   0                    0                    0 
           Pubs_Bars        LocalServices    Burger_PizzaShops 
                   0                    1                    1 
Hotels_OtherLodgings            JuiceBars         ArtGalleries 
                   0                    0                    0 
          DanceClubs       Swimming.Pools                 Gyms 
                   0                    0                    0 
            Bakeries           BeautySpas                Cafes 
                   0                    0                    0 
          ViewPoints            Monuments              Gardens 
                   0                    0                    1 
               ...26 
                5454

Churches              Resorts              Beaches 
                   0                    0                    0 
               Parks             Theatres              Museums 
                   0                    0                    0 
               Malls                  Zoo          Restaurants 
                   0                    0                    0 
           Pubs_Bars        LocalServices    Burger_PizzaShops 
                   0                    1                    1 
Hotels_OtherLodgings            JuiceBars         ArtGalleries 
                   0                    0                    0 
          DanceClubs       Swimming.Pools                 Gyms 
                   0                    0                    0 
            Bakeries           BeautySpas                Cafes 
                   0                    0                    0 
          ViewPoints            Monuments              Gardens 
                   0                    0                    0 
               ...26 
                5454

    Churches        Resorts         Beaches          Parks      
 Min.   :0.000   Min.   :0.000   Min.   :0.000   Min.   :0.830  
 1st Qu.:0.920   1st Qu.:1.360   1st Qu.:1.540   1st Qu.:1.730  
 Median :1.340   Median :1.905   Median :2.060   Median :2.460  
 Mean   :1.456   Mean   :2.320   Mean   :2.489   Mean   :2.797  
 3rd Qu.:1.810   3rd Qu.:2.683   3rd Qu.:2.740   3rd Qu.:4.093  
 Max.   :5.000   Max.   :5.000   Max.   :5.000   Max.   :5.000  
                                                                
    Theatres        Museums          Malls            Zoo       
 Min.   :1.120   Min.   :1.110   Min.   :1.120   Min.   :0.860  
 1st Qu.:1.770   1st Qu.:1.790   1st Qu.:1.930   1st Qu.:1.620  
 Median :2.670   Median :2.680   Median :3.230   Median :2.170  
 Mean   :2.959   Mean   :2.893   Mean   :3.351   Mean   :2.541  
 3rd Qu.:4.312   3rd Qu.:3.840   3rd Qu.:5.000   3rd Qu.:3.190  
 Max.   :5.000   Max.   :5.000   Max.   :5.000   Max.   :5.000  
                         

[1] 5456   25

Churches              Resorts              Beaches 
                 195                   90                    4 
               Parks             Theatres              Museums 
                   0                    0                    0 
               Malls                  Zoo          Restaurants 
                   0                    0                    0 
           Pubs_Bars        LocalServices    Burger_PizzaShops 
                   0                   NA                   NA 
Hotels_OtherLodgings            JuiceBars         ArtGalleries 
                   0                    0                    4 
          DanceClubs       Swimming.Pools                 Gyms 
                 112                  479                 1017 
            Bakeries           BeautySpas                Cafes 
                1046                  896                  604 
          ViewPoints            Monuments              Gardens 
                 345                  302                  225 
               ...26 
                  NA

In [5]:
# #####Run full correlation matrix####
# #Prep functions
# get_upper_tri <- function(mcor){
#   mcor[lower.tri(mcor)]<- NA
#   return(mcor)
# }
# 
# reorder_mcor <- function(mcor){
#   # Use correlation between variables as distance
#   dd <- as.dist((1-mcor)/2)
#   hc <- hclust(dd)
#   mcor <-mcor[hc$order, hc$order]
# }
# 
# #Construct text
# mcor <- cor(travelData)
# melted_mcor <- melt(mcor)
# mcor <- reorder_mcor(mcor)
# upper_tri <- get_upper_tri(mcor)
# 
# # Melt the correlation matrix
# melted_mcor <- melt(upper_tri, na.rm = TRUE)
# # Create a ggheatmap
# ggheatmap <- ggplot(melted_mcor, aes(Var2, Var1, fill = value))+
#   ggtitle("Pearson Correlation Analysis of Google Travel Variables") +
#   geom_tile(color = "white")+
#   scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
#                        midpoint = 0, limit = c(-1,1), space = "Lab", 
#                        name="Pearson\nCorrelation") +
#   theme_minimal()+ # minimal theme
#   theme(axis.text.x = element_text(angle = 90, vjust = 1, 
#                                    size = 9, hjust = 1)
#         ,
#         plot.title = element_text(size = 18))+
#   coord_fixed() +
#   geom_text(aes(Var2, Var1, label = round(value,2)), color = "black", size = 4) +
#   theme(
#     axis.title.x = element_blank(),
#     axis.title.y = element_blank(),
#     panel.grid.major = element_blank(),
#     panel.border = element_blank(),
#     panel.background = element_blank(),
#     axis.ticks = element_blank(),
#     legend.justification = c(1, 0),
#     legend.position = c(0.6, 0.7),
#     legend.direction = "horizontal")+
#   guides(fill = guide_colorbar(barwidth = 7, barheight = 1,
#                                title.position = "top", title.hjust = 0.5))
# # Print the heatmap
# print(ggheatmap)
# 
# #Looks like theres a correlation between Theatres and Parks - what does that look like?
# ggplot(travelData, aes(Theatres, Parks)) + geom_point() 
# 
# 
# #5s look like they may be a problem
# #### BIG histogram ####
# travelDataLongdf <- reshape(travelData, varying = names(travelData), v.names = "value", 
#                             times = names(travelData), timevar = "variable", direction = "long")
# 
# ggplot(travelDataLongdf, aes(x = value, y = -0.5, xlim = 5)) +
#   
#   # horizontal box plot
#   geom_boxplot() +
#   
#   # normal density plot
#   geom_density(aes(x = value), inherit.aes = FALSE) +
#   xlim(0, 5) +
#   facet_wrap(variable ~ .) +
#   
#   # reproduce original chart's color scale (o/w ordered factors will result
#   # in viridis scale by default, using the current version of ggplot2)
#   scale_fill_discrete()
# 
# #looks like there's definitely a skew towards 5 and 1/0 in a lot of these variables
# #perhaps split up the varaibles into polarizing and nonpolarizing?
# 
# #### PCA ####
# set.seed(123)
# travel_pca <- prcomp(travelData, 
#                      center = TRUE, 
#                      scale. = TRUE)
# 
# var = get_pca_var(travel_pca)
# 
# #variable loadings (i.e. eigenvectors)
# travel_pca$rotation
# 
# #PC values/scores for all observations (i.e., projected values onto principal component axes)
# travel_pca$x 
# 
# #Obtain eigenvalues
# get_eig(travel_pca) 
# 
# #Visualize explained variances per component
# fviz_eig(travel_pca, addlabels=TRUE) 
# 
# #Quality representation :
# corrplot(var$cos2,col = colorRampPalette(c("white", "deepskyblue", "blue4"))(100), is.corr=FALSE)
# 
# #Interesting! 7 eigenvalues 
# 
# 
# var$coord #coordinates of each variable
# var$cos2 #cos2 of each variable
# var$contrib #contributions of each variable to each component
# 
# sevenContributions <- data.frame(var$contrib[,1:7])
# 
# sevenContributions <- rownames_to_column(sevenContributions, "alpha")
# 
# 
# #### PCA2 ####
# travelData2 <- travelData %>%
#   select(c(-Bakeries,-Gyms,-BeautySpas,-Parks,-Theatres, -Zoo, -Cafes, -Swimming.Pools))
# 
# set.seed(123)
# travel_pca2 <- prcomp(travelData2, center = TRUE, scale. = TRUE)
# 
# var = get_pca_var(travel_pca2)
# 
# #variable loadings (i.e. eigenvectors)
# travel_pca2$rotation
# 
# #PC values/scores for all observations (i.e., projected values onto principal component axes)
# #saving results for clustering
# pca2_results = travel_pca2$x 
# 
# #Obtain eigenvalues
# get_eig(travel_pca2) 
# 
# #Visualize explained variances per component
# fviz_eig(travel_pca2, addlabels=TRUE) 
# 
# #Quality representation :
# corrplot(var$cos2,col = colorRampPalette(c("white", "deepskyblue", "blue4"))(100), is.corr=FALSE)
# 
# #A little better with only 5 dimensions
# 
# var$coord #coordinates of each variable
# var$cos2 #cos2 of each variable
# var$contrib #contributions of each variable to each component
# 
# #### T-SNE (didnt work) ####
# # set.seed(123)
# # travelData_s <- scale(travelData)
# # travel_tsne <- tsne(travelData_s, 
# #      initial_config = NULL, 
# #      k = 2, 
# #      initial_dims = 2, 
# #      perplexity = 30,
# #      max_iter = 1000, 
# #      min_cost = 0, 
# #      epoch_callback = NULL, 
# #      whiten = TRUE,
# #      epoch=100)
# # 
# # plot(travel_tsne$Y, main="tsne")
# 
# 
# 
# #### Independent Component Analysis (ICA) (also didnt work) ####
# # set.seed(123)
# # travel_ica <- fastICA(travelData, 
# #                       n.comp = 3,
# #                       alg.typ = "deflation",
# #                       fun = "logcosh",
# #                       method = "R",
# #                       row.norm = TRUE,
# #                       maxit = 150000,
# #                       alpha = 1,
# #                       tol = 0.0001,
# #                       verbose = TRUE)
# # 
# # plot(travel_ica$X, main = "Pre-Processed Data")
# # plot(travel_ica$X %*% travel_ica$K, main = "PCA Components")
# # plot(travel_ica$S, main = "ICA Components")
# 
# #### Principal Coordinates Analysis PCoA (also didnt work) :( ####
# # library(ecodist)
# # 
# # set.seed(123)
# # travelData.bray <- vegdist(travelData, method = "bray") # dissimilarity matrix using bray-curtis distance indices on the varespec dataset native to vegan
# # 
# # travel_PCoA <- pco(vtravelData.bray, negvals = "zero", dround = 0) # if negvals = 0 sets all negative eigenvalues to zero; if = "rm" corrects for negative eigenvalues using method 1 of Legendre and Anderson 1999
# # 
# # 
# # plot(travel_PCoA$vectors[,1], travel_PCoA$vectors[,2], type = "n", xlab = "PCoA1", ylab = "PCoA2",
# #      axes = TRUE, main = "PCoA (ecodist) on varespec data")
# # 
# # text(travel_PCoA$vectors[,1], travel_PCoA$vectors[,2], labels(varespec.bray), 
# #      cex = 0.9, xpd = TRUE)
# # 
# # travel_PCoA$values # eigenvalue for each component. This is a measure of the variance explained by each dimension
# # travel_PCoA$vectors # eigenvectors. Each column contains the scores for that dimension
# 
# #############################################################################
# 
# #### k-Means Clustering ##### 
# #### using data as it with 24 variables####
# #### to get the optimal number of clusters###
# 
# #Elbow method - with maximum 10 clusters
# elb_m_1 = fviz_nbclust(travelData, kmeans, k.max=10, nstart=50, method="wss") + ggtitle("Optimal number of clusters", subtitle = "K-Means 24 variables - Elbow Method")
# ### 3-5 cluster as optimal # of clusters
# 
# #Silhouette method - maximum 10 clusters
# sil_m_1 =fviz_nbclust(travelData, kmeans, k.max=10, nstart=50, method="silhouette")+ ggtitle("Optimal number of clusters", subtitle = "K-Means 24 variables - Silhouette Method")
# ### silouette method returned 9 as the optimal number of clusters
# 
# ### getting distance data
# dist_one = get_dist(travelData)
# 
# #### visualizing distance between datapoints 
# fviz_dist(dist_one, gradient = list(low = "blue", mid = "yellow", high = "darkorange"))
# 
# ### kmeans -- setting center to value of 3 based on elbow method
# set.seed(2023)
# k_clus = kmeans(travelData, centers = 3, nstart = 50)
# 
# k_clus$cluster
# ### centroids - average of all the variables in each cluster - standardized (z-score)
# k_clus$centers
# ### number of obs in each clusters
# k_clus$size
# 
# #### to analyze/visualize similar an object is to the other objects in its own cluster versus 
# #### those in its neighbor cluster 
# sil = silhouette(k_clus$cluster, dist(travelData)) 
# prop.table(sil)
# sil_plot = fviz_silhouette(sil)
# 
# #### comparing K-means results to first two dims in PCA
# fviz_cluster(k_clus, data =travelData)
# 
# ############ exploring # of k values 4&5 based on elbow results #############
# 
# ########### k-means using four centers ##########
# set.seed(2023)
# k_clus_4c = kmeans(travelData, centers = 4, nstart = 50)
# 
# ### reviewing distance from center
# k_clus_4c$centers
# 
# ### number of obs in each clusters
# k_clus_4c$size
# 
# ### clusters vs neighbor cluster
# sil_two = silhouette(k_clus_4c$cluster, dist(travelData))
# prop.table(sil_two)
# fviz_silhouette(sil_two)
# 
# #### comparing K-means results to first two dims in PCA
# fviz_cluster(k_clus_4c, data =travelData)
# 
# ############# k-means using five centers ############### 
# set.seed(2023)
# k_clus_5c = kmeans(travelData, centers = 5, nstart = 50)
# 
# ### number of obs in each clusters
# k_clus_5c$size
# 
# ### clusters vs neighbor cluster
# sil_5c = silhouette(k_clus_5c$cluster, dist(travelData))
# prop.table(sil_5c)
# fviz_silhouette(sil_5c)
# 
# ### cluster output via dim1 & dim 2
# fviz_cluster(k_clus_5c, data =travelData)
# 
# ################## cluster comparison ########################
# p1 = fviz_cluster(k_clus, geom = "point", data = travelData) + ggtitle("k = 3, 24 Vars")
# p2 = fviz_cluster(k_clus_4c, geom = "point",  data = travelData) + ggtitle("k = 4, 24 Vars")
# p3 = fviz_cluster(k_clus_5c, geom = "point",  data = travelData) + ggtitle("k = 5, 24 Vars")
# 
# #### three clusters seems to best represent the data ####
# library(gridExtra)
# grid.arrange(p1, p2, p3, nrow = 3)
# ?grid.arrange()
# 
# #################### K means --> using 16 variables #####################
# 
# ### elbow method still shows 3 as optimal number of clusters
# fviz_nbclust(travelData2, kmeans, method = 'wss') + ggtitle("Optimal number of clusters", subtitle = "K-Means 16 variables - Elbow Method")
# 
# #Silhouette method - maximum 10 clusters
# fviz_nbclust(travelData2, kmeans, k.max=10, nstart=50, method="silhouette")+ ggtitle("Optimal number of clusters", subtitle = "K-Means 16 variables - Silhouette Method")
# 
# 
# set.seed(2023)
# km2_clus = kmeans(travelData2, centers = 3, nstart = 50)
# 
# ### saving results 
# p_km2_clus = fviz_cluster(km2_clus, geom = "point",data = travelData2) + ggtitle("k = 3, 16 vars")
# 
# 
# ### silhouette results 
# sil_2 = silhouette(km2_clus$cluster, dist(travelData2))
# prop.table(sil_2)
# sil_plot_2 = fviz_silhouette(sil_2)
# 
# ### comparing kmeans 24 vars to  pca2 kmeans 16 vars
# grid.arrange(p1, p_km2_clus, nrow = 2)
# ### K-means with 16 variables Dims 1&2 show better results because % is slightly higher
# 
# #### silhouette plot comparison to identify which model better classified clusters
# grid.arrange(sil_plot, sil_plot_2, nrow = 1)
# #### average silhouette width is 0.01 higher in sil_2 (km2_clus)
# 
# #### created a bar plot from the $clus values to see the presence of each variable by cluster ####
# barplot(km2_clus$centers, las =2, border = F,beside = TRUE, legend  = TRUE
#         col=c("coral1" , "skyblue1" , "palegreen3"), main ="Variable Contributions by K-Means Cluster")
# 
# ########################################################################################
# #### HIERARCHICAL CLUSTERING WITHOUT SCALED DATA######
# 
# prox_matrix = dist(travelData, method = "euclidean")
# 
# #best number for hierarchical (based on 30 indices)
# #(this can take long for large datasets, going up to 7 clusters to save running time)
# nb_hc_c = NbClust(data = travelData, distance = "euclidean", method = "complete",
#                   min.nc = 2, max.nc = 7) #The number of recomended clusters changed from 4 to 7 now without scaled data
# 
# nb_hc_w = NbClust(data = travelData, distance = "euclidean", method = "ward.D2",
#                   min.nc = 2, max.nc = 7) #Number of cluders recomended is 3 now as well when data is not scaled
# 
# 
# hc_complete = hclust(prox_matrix, method = "complete")
# hc_ward = hclust(prox_matrix, method = "ward.D2")
# 
# # Cutting dendrogram to specific number of clusters
# cut_hc_complete = cutree(hc_complete, k = 7) #Switched from 4 to 7 because of number of clusters
# cut_hc_ward = cutree(hc_ward, k = 3)
# 
# 
# #Check the number of observations in each cluster. 
# #Do they look balanced? 
# # The ward table looks mostly balanced but the complete table is not balanced at all
# table(cut_hc_complete)
# table(cut_hc_ward)
# 
# # Attach cluster membership info to the original dataset and compare variable averages of each group
# travel_info = cbind(df, cluster_c = cut_hc_complete, cluster_w = cut_hc_ward)
# ### getting distance to apply to hclust
# distance = get_dist(travelData)
# hclus = hclust(d = distance, method = "complete")
# 
# #Cutting the tree into five groups
# cut_hc = cutree(hclus, k = 3) #Using 3 because that was a close 2nd and 7 will be too many clusters
# 
# 
# # Attach cluster membership info to the original dataset
# travel_clus_h = cbind(travelData, cluster = cut_hc)
# 
# 
# #This code actually gives you values
# aggregate(travel_clus_h, by = list(travel_clus_h$cluster), mean)
# aggregate(travel_info, by = list(travel_info$cluster_c), mean)  #Cluster centroids (complete linkage method)
# aggregate(travel_info, by = list(travel_info$cluster_w), mean) #Cluster centroids (ward distance method)
# 
# # Cut in 5 groups and color by groups
# fviz_dend(hclus, k = 3, 
#           cex = 0.5, # label size
#           k_colors = c("#2E9FDF", "#E7B800", "#FC4E07"),
#           color_labels_by_k = TRUE, # color labels by groups
#           rect = TRUE) # Add rectangle around groups
# 
# fviz_dend(hc_ward, k = 3, 
#           cex = 0.5, # label size
#           k_colors = c("#2E9FDF", "#00AFBB", "#E7B800", "#FC4E07", "#00FF00"),
#           color_labels_by_k = TRUE, # color labels by groups
#           rect = TRUE) # Add rectangle around groups
# 
# #Visualize results in a scatter plot
# fviz_cluster(list(data = df, cluster = cut_hc),
#              palette = c("#2E9FDF", "#E7B800", "#FC4E07"),
#              ellipse.type = "convex", # Concentration ellipse
#              repel = FALSE, # Allow label overplotting (slow)
#              show.clust.cent = FALSE)
# 
# fviz_cluster(list(data = travelData, cluster = hc_ward),
#              palette = c("#2E9FDF", "#00AFBB", "#E7B800", "#FC4E07", "#00FF00"),
#              ellipse.type = "convex", # Concentration ellipse
#              repel = FALSE, # Allow label overplotting (slow)
#              show.clust.cent = FALSE)
